# Solving a Sudoku with AI

## Goals of this project

AI is composed of simple ideas put together to solve complex problems. Two of these ideas are **constraint propagation** and **search**.

### Contraint Propagation

* Each square in sudoku has **local constraints**
* Maximum information is extracted from each constraint
* Simple constraints applied iteratively to narrow search space of solutions
* Constraint propagation can be used to solve problems like calendar scheduling and cryptographic puzzles
* Note: Logic programming languages like [Prolog][] are pretty much made for **constraint satisfaction problems**

[Prolog]: http://www.swi-prolog.org/

### Search

- While solving a problem, there may be a point where more than one possible solutions exist
- These branching points culminate in a tree of solutions
- The tree can be traversed to find a solution

## Setting up the Board

- **boxes**: Individual squares (e.g. 'A1', 'A2', ..., 'I9')
- **units**: Row, column, or 3x3 square
- **peers**: All other boxes that belong to the same unit as a box (the boxes in the row, column, and 3x3 square)

Any given box will have 20 peers: 8 in the 3x3 square, 6 _additional_ boxes per row, and 6 _additional_ boxes per column.

## Coding the Board

In [4]:
rows = 'ABCDEFGHI'
cols = '123456789'

In [5]:
def cross(a, b):
    return [
        s + t
        for s in a
        for t in b
    ]

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [6]:
boxes = cross(rows, cols)

In [31]:
row_units = [
    cross(r, cols)
    for r in rows
]
column_units = [
    cross(rows, c)
    for c in cols
]
square_units = [
    cross(rs, cs)
    for rs in ('ABC', 'DEF', 'GHI')
    for cs in ('123', '456', '789')
]
unitlist = row_units + column_units + square_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)
print('row example', row_units[0])
print('column example', column_units[0])
print('square example', square_units[0])
print('unit example', units.items()[0])
print('peer example', peers.items()[0])

('row example', ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'])
('column example', ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'])
('square example', ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3'])
('unit example', ('I6', [['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9'], ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6'], ['G4', 'G5', 'G6', 'H4', 'H5', 'H6', 'I4', 'I5', 'I6']]))
('peer example', ('I6', set(['I9', 'I8', 'G5', 'G4', 'F6', 'G6', 'I1', 'I5', 'I3', 'I2', 'H6', 'I4', 'I7', 'H5', 'B6', 'H4', 'A6', 'D6', 'E6', 'C6'])))


In [39]:
EXAMPLE_BOARD = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'

In [40]:
import re

def grid_values(grid):
    # In this function, you will take a sudoku as a string
    # and return a dictionary where the keys are the boxes,
    # for example 'A1', and the values are the digit at each
    # box (as a string) or '.' if the box has no value
    # assigned yet.
    assert re.compile(r'^[1-9.]{81}$').match(grid)
    return dict(zip(boxes, grid))

display(grid_values(EXAMPLE_BOARD))

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


## Strategy 1: Elimination

If a box has a value assigned, then none of the peers of this box can have this value.

In [41]:
import re

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    assert re.compile(r'^[1-9.]{81}$').match(grid)
    all_nums = '123456789'
    boxes_and_values = zip(boxes, grid)
    return {
        box: all_nums if value == '.' else value
        for box, value in boxes_and_values
    }

display(grid_values(EXAMPLE_BOARD))

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

In [42]:
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    result = values.copy() # don't mutate the original values dict
    final_values = get_final_values(result)
    for box, value in final_values:
        peers_for_box = peers[box]
        eliminate_from_peers(result, peers_for_box, value)
    return result

def get_final_values(values):
    return [
        (box, value)
        for box, value in values.items()
        if is_final(value)
    ]

def is_final(value):
    return len(value) == 1
    
def eliminate_from_peers(values, peers, num_to_eliminate):
    for peer in peers:
        old_value = values[peer]
        if not is_final(old_value):
            new_value = old_value.replace(num_to_eliminate, '')
            values[peer] = new_value

display(eliminate(grid_values(EXAMPLE_BOARD)))

   45    4578    3   |   49     2     147  |   6     5789    57  
   9    24678    47  |   3      47     5   |   78    278     1   
   25    257     1   |   8      79     6   |   4    23579   2357 
---------------------+---------------------+---------------------
  345    345     8   |   1     3456    2   |   9    34567  34567 
   7    123459   49  |  459   34569    4   |   1    13456    8   
  1345  13459    6   |   7     3459    8   |   2     1345   345  
---------------------+---------------------+---------------------
  134    1347    2   |   6     478     9   |   5     1478    47  
   8     1467    47  |   2     457     3   |   17    1467    9   
   46    4679    5   |   4      1      47  |   3    24678   2467 


## Strategy 2: Only Choice

If there is only one box in a unit which would allow a cetrain digit, then that box must be assigned that digit.

In [45]:
from collections import Counter

def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    new_values = values.copy()  # note: do not modify original values
    for unit in unitlist:
        # Only relying on initial state to satisfy grader; however,
        # it would be more efficient to iterate using the new state.
        only_choices = get_only_choices_for_unit(values, unit)
        assign_values(new_values, only_choices)
    return new_values

def get_only_choices_for_unit(values, unit):
    possibility_counts = Counter()
    # Maintain an index of value -> box
    # Since we only care about only choices, allow the box to be overwritten
    # Only choices will inherently not be overwritten.
    value_to_box = dict()
    # Count appearances of values and create index of value -> last box
    for box in unit:
        box_values = list(values[box])
        possibility_counts.update(box_values)
        for value in box_values:
            existing_values = value_to_box[value] = box
    return [
        (value_to_box[value], value)
        for value, count in possibility_counts.items()
        if count == 1
    ]

def assign_values(values, boxes_and_values):
    for box, value in boxes_and_values:
        values[box] = value
        
display(only_choice(eliminate(grid_values(EXAMPLE_BOARD))))

  45   4578   3   |  49    2     1   |  6    5789   57  
  9     6     47  |  3     47    5   |  8    278    1   
  2    257    1   |  8     79    6   |  4   23579  2357 
------------------+------------------+------------------
 345   345    8   |  1    3456   2   |  9   34567 34567 
  7     2     9   |  5   34569   4   |  1   13456   8   
 1345 13459   6   |  7    3459   8   |  2    1345  345  
------------------+------------------+------------------
 134   1347   2   |  6     8     9   |  5    1478   47  
  8    1467   47  |  2     5     3   |  17   1467   9   
  6     9     5   |  4     1     47  |  3     8    2467 


## Constraint Propagation

Constraint propagation is the iterative application of local constraints to reduce the overall search space.

### Constraint Propagation in Sudoku

In Sudoku, this means starting with the initial grid state, and the applying the `eliminate` and `only_choice` constraints repeatedly until arriving at a solution or stalling (no progress being made).

The algorithm can roughly be described by the following (this is generally applicable):
1. Store the in-progress solution before applying constraints.
1. Apply all constraints.
1. Compare the new in-progress solution with the solution before applying constraints. If there is no change, set state to stalled.
1. Repeat while state is not stalled.

![Constraint Propagation in Sudoku](constraint-propagation.png)

### Other Examples of Constraint Propagation Problems

- Map coloring: Color a map where no two adjacent regions are the same color.
- Crypto-arithmetic puzlzes: Find a mapping where each letter maps to a digit. Given a set of equations, find a mapping that satisfies them.

In [47]:
def reduce_puzzle(values):
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Your code here: Use the Eliminate Strategy
        values = eliminate(values)

        # Your code here: Use the Only Choice Strategy
        values = only_choice(values)

        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

### Harder Sudoku

Some Sudoku problems cannot be solved in a single round of constraint propagation. Instead, the solution space needs to be searched after constraint propagation. Below is an example of a harder Sudoku puzzle after going through a round of constraint propagation.

In [52]:
EXAMPLE_BOARD2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
display(reduce_puzzle(grid_values(EXAMPLE_BOARD2)))

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


## Strategy 3: Search

In cases where constraint propagation do not lead to one or no solutions, search can be used to efficiently find solutions. The algorithm can roughly be described as a [depth-first search][DFS], creating the minimum number of branches at each step.

Pick a box with a minimal number of possible values. Try to solve each of the puzzles obtained by choosing each of these values, recursively.

<!-- Links -->
[DFS]: https://en.wikipedia.org/wiki/Depth-first_search

In [63]:
def search(values):
    """
    Using depth-first search and propagation, create a search tree and solve the sudoku.
    NOTE: This assumes that we only want A solution, not ALL solutions.
    """
    # First, reduce the puzzle using the previous function
    reduced_values = reduce_puzzle(values)
    
    # Base case when puzzle is unsolvable
    if reduced_values is False:
        return False
    
    # Choose one of the unfilled squares with the fewest possibilities
    min_choice = get_box_with_fewest_possibilities(reduced_values)

    # Base case when puzzle is already solved
    if min_choice is None:
        return reduced_values
    
    min_box, possible_values = min_choice
    
    # Now use recursion to solve each one of the resulting sudokus, and if one returns a value (not False), return that answer!
    for possible_value in list(possible_values):
        subtree_values = reduced_values.copy()
        subtree_values[min_box] = possible_value
        subtree_solution = search(subtree_values)
        if subtree_solution is not False:
            return subtree_solution
    
    # Unsolvable puzzle
    return False

def get_box_with_fewest_possibilities(values):
    unsolved_values = [
        (box, value)
        for box, value in values.items() if len(value) > 1
    ]
    if len(unsolved_values) == 0:
        return None
    sorted_values = sorted(unsolved_values, key=lambda x: len(x[1]))
    return sorted_values[0]

In [64]:
display(search(grid_values(EXAMPLE_BOARD2)))

4 1 7 |3 2 6 |8 9 5 
6 3 8 |1 9 5 |7 4 7 
2 5 9 |7 3 8 |2 1 6 
------+------+------
8 2 5 |4 9 7 |9 6 1 
7 9 1 |5 8 6 |4 2 3 
3 4 6 |9 1 2 |7 5 8 
------+------+------
9 8 2 |6 5 3 |1 7 4 
5 7 3 |2 4 1 |6 8 9 
1 6 4 |8 7 9 |5 3 2 
